In [3]:
using Pkg
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/Origin.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/Deformula.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/SparseMatrix.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/NMarkov.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/SymbolicDiff.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/SymbolicMarkov.jl.git"))
# Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/PhaseTypeDistributions.jl.git"))

    Updating git-repo `https://github.com/JuliaReliab/Origin.jl.git`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.6/Project.toml`
  No Changes to `/opt/julia/environments/v1.6/Manifest.toml`
    Updating git-repo `https://github.com/JuliaReliab/Deformula.jl.git`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.6/Project.toml`
  No Changes to `/opt/julia/environments/v1.6/Manifest.toml`
    Updating git-repo `https://github.com/JuliaReliab/SparseMatrix.jl.git`
   Resolving package versions...


LoadError: Unsatisfiable requirements detected for package [38;5;2mSparseMatrix [b2a93422][39m:
 [38;5;2mSparseMatrix [b2a93422][39m log:
 ├─possible versions are: [38;5;2m0.3.3[39m or uninstalled
 ├─restricted to versions [38;5;2m0.2.3-0.2[39m by [38;5;11mPhaseTypeDistributions [6af5a0ae][39m — no versions left
 │ └─[38;5;11mPhaseTypeDistributions [6af5a0ae][39m log:
 │   ├─possible versions are: [38;5;11m0.2.6[39m or uninstalled
 │   └─[38;5;11mPhaseTypeDistributions [6af5a0ae][39m is fixed to version [38;5;11m0.2.6[39m
 └─[38;5;2mSparseMatrix [b2a93422][39m is fixed to version [38;5;2m0.3.3[39m

In [5]:
using SparseArrays
using NMarkov
using SymbolicDiff
using SymbolicMarkov
using Plots
using Distributions
using Random

┌ Info: Precompiling NMarkov [c5cca998-98d3-11ea-1288-bb4300628a81]
└ @ Base loading.jl:1342
┌ Warning: Module JLLWrappers with build ID 14418454654031536 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1030
┌ Warning: Module JLLWrappers with build ID 14418454654031536 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1030
┌ Warning: Module JLLWrappers with build ID 14418454654031536 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1030
┌ Warning: Module JLLWrappers with build ID 14418454654031536 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3

In [4]:
Pkg.update()

    Updating registry at `/opt/julia/registries/General`
    Updating git-repo `https://github.com/JuliaReliab/FaultTree.jl.git`
    Updating git-repo `https://github.com/JuliaReliab/SparseMatrix.jl.git`
    Updating git-repo `https://github.com/JuliaReliab/SymbolicMarkov.jl.git`
    Updating git-repo `https://github.com/JuliaReliab/Deformula.jl.git`
    Updating git-repo `https://github.com/JuliaReliab/Origin.jl.git`
    Updating git-repo `https://github.com/JuliaReliab/NMarkov.jl.git`
    Updating git-repo `https://github.com/JuliaReliab/PhaseTypeDistributions.jl.git`
    Updating git-repo `https://github.com/JuliaReliab/SymbolicDiff.jl.git`
    Updating git-repo `https://github.com/JuliaReliab/JuliaDot.jl.git`
    Updating git-repo `https://github.com/JuliaReliab/DD.jl.git`
   Installed PlotUtils ─────────────── v1.3.0
   Installed SpecialFunctions ──────── v2.1.6
   Installed IJulia ────────────────── v1.23.3
   Installed StatsBase ─────────────── v0.33.18
   Installed Plots ──────

# Example: SIFT-like system

- p.184 in Red book

### SHARPE code

```
* SIFT example

func c(x, k) \
  360/(360 + (k-1) * x)
func r(x,k) \
  k * x * c(x,k)
func f(x,k) \
  k * x * (1-c(x,k))

markov SIFT
4-3 3-3 r(lambda,4)
3-3 2-3 r(lambda,3)

4-3 F f(lambda,4) + f(mu,3)
3-3 F f(lambda,3) + f(mu,3)
2-3 F (2 * lambda) + f(mu,3)

4-3 4-2 r(mu,3)
3-3 3-2 r(mu,3)
2-3 2-2 r(mu,3)

4-2 3-2 r(lambda,4)
3-2 2-2 r(lambda,3)

4-2 F (2 * mu) + f(lambda,4)
3-2 F (2 * mu) + f(lambda,3)
2-2 F (2 * mu) + (2 * lambda)
end
end

bind
lambda .0001
mu .00001
end

expr mean(SIFT,F)
expr variance(SIFT,F)
expr value(10;SIFT,F)
end
```

In [7]:
@macroexpand @markov SIFT(;lambda, mu) begin
    c(x,k) = 360/(360 + (k-1) * x)
    r(x,k) = k * x * c(x,k)
    f(x,k) = k * x * (1-c(x,k))
    @tr begin
        @s("4-3") => @s("3-3"), r(lambda,4)
        @s("3-3") => @s("2-3"), r(lambda,3)

        @s("4-3") => @s("F"), f(lambda,4) + f(mu,3)
        @s("3-3") => @s("F"), f(lambda,3) + f(mu,3)
        @s("2-3") => @s("F"), 2 * lambda + f(mu,3)

        @s("4-3") => @s("4-2"), r(mu,3)
        @s("3-3") => @s("3-2"), r(mu,3)
        @s("2-3") => @s("2-2"), r(mu,3)

        @s("4-2") => @s("3-2"), r(lambda,4)
        @s("3-2") => @s("2-2"), r(lambda,3)

        @s("4-2") => @s("F"), (2 * mu) + f(lambda,4)
        @s("3-2") => @s("F"), (2 * mu) + f(lambda,3)
        @s("2-2") => @s("F"), (2 * mu) + (2 * lambda)
    end
    @init begin
        @s("4-3"), 1
    end
    @reward :down begin
        @s("F"), 1
    end
end


:(function SIFT(; lambda, mu)
      tmp = Markov()
      #= In[7]:2 =#
      c(x, k) = begin
              #= In[7]:2 =#
              360 / (360 + (k - 1) * x)
          end
      #= In[7]:3 =#
      r(x, k) = begin
              #= In[7]:3 =#
              k * x * c(x, k)
          end
      #= In[7]:4 =#
      f(x, k) = begin
              #= In[7]:4 =#
              k * x * (1 - c(x, k))
          end
      #= In[7]:5 =#
      begin
          #= In[7]:6 =#
          trans!(tmp, Symbol("4-3"), Symbol("3-3"), r(lambda, 4))
          #= In[7]:7 =#
          trans!(tmp, Symbol("3-3"), Symbol("2-3"), r(lambda, 3))
          #= In[7]:9 =#
          trans!(tmp, Symbol("4-3"), Symbol("F"), f(lambda, 4) + f(mu, 3))
          #= In[7]:10 =#
          trans!(tmp, Symbol("3-3"), Symbol("F"), f(lambda, 3) + f(mu, 3))
          #= In[7]:11 =#
          trans!(tmp, Symbol("2-3"), Symbol("F"), 2lambda + f(mu, 3))
          #= In[7]:13 =#
          trans!(tmp, Symbol("4-3"), Symbol("4-2"), r(mu, 3)

In [3]:
begin
    lambda = 0.0001
    mu = 0.00001
end

1.0e-5

In [4]:
# compute
#   mean(SIFT,F)
#   variance(SIFT,F)
#   value(10; SIFT, F)
#

# gen CTMC and PH
sift = ctmc(SIFT(lambda = lambda, mu = mu))
up = [x != 1 for x = sift.reward[:down]]
T = sift.Q[up,up]
alpha = sift.initv[up]
tmp = (-T)' \ alpha
siftm = sum(tmp)
tmp = (-T)' \ tmp
siftv = sum(tmp) * 2 - siftm^2
sift10 = exrt(10.0, sift, reward=:down)[1]
(siftm, siftv, sift10)

(10604.190777840504, 3.990485188959943e7, 3.7273559466670164e-8)

# Example: The ten processor example

- p.187 in Red book

### SHARPE code

```
markov 10proc(lam, mu, c)
...
end
```

In [41]:
@markov model10proc(;lam, mu, c, K) begin
    for i = K:-1:2
        @tr @s(i) => @s(i-1), i*lam*c
        @tr @s(i) => @s(0), i*lam*(1-c)
    end
    @tr @s(1) => @s(0), lam
    for i = 1:K-1
        @tr @s(i) => @s(i+1), mu
    end
    @init @s(K), 1
    @reward :a begin
        @s(K), 1
    end
end

model10proc (generic function with 2 methods)

In [42]:
@bind begin
    lam = 1.0
    mu = 1.0
    c = 0.9999
end
K = 10

10

In [43]:
tenproc = ctmc(model10proc(lam = lam, mu = mu, c = c, K = K))

"CTMCModel(2926725442712203260)"

In [45]:
seval(exrt(LinRange(0, 10, 100), tenproc, reward = :a))

100-element Vector{Float64}:
 1.0
 0.3830645465949663
 0.16153107553772378
 0.07404714116313584
 0.03635225553799549
 0.018854421983638556
 0.01022038743323745
 0.005744040314660446
 0.0033277017617644494
 0.0019788839202512535
 0.0012042087875139811
 0.0007481430895357245
 0.0004737029660688018
 ⋮
 2.0753109160150083e-8
 1.982701344306784e-8
 1.8942601136225316e-8
 1.809793914490598e-8
 1.7291193033922013e-8
 1.6520620606148166e-8
 1.5784566093255882e-8
 1.508145487881577e-8
 1.4409788685997998e-8
 1.3768141172237441e-8
 1.315515388175961e-8
 1.2569532514045706e-8